# Welcome to NLP 101!
This notebook will be covering both classical NLP and deep learning NLP techniques.
Let's import some packages that we will be using first. We will be first using nltk - the natural language toolkit to show us some classical NLP capabilities. These are important as we can still use these techniques in conjunction with our ML/DL models as preprocessing steps.

In [ ]:
##bit of change for git practice
import nltk
nltk.download('omw-1.4')

## A. Classical NLP
Let's define an input paragraph that we want to analyze first!

In [ ]:
# Feel free to change the text - this is from Wikipedia
text = "Tower Bridge is a drawbridge in London. It crosses the River Thames near the Tower of London. It allows ships through the bridge deck when is raised at an angle in the centre. The north side of the bridge is Tower Hill, and the south side of the bridge comes down into Bermondsey, an area in Southwark. Tower Bridge is far more visible than London Bridge, which people often mistake it for. Many tourists go to London to see the Tower Bridge. It has its own exhibition centre in the horizontal walkway. This gives one of the best vantage points in London."

### 1. Sentence segmentation
Split the sentences up!

In [ ]:
sentences = nltk.sent_tokenize(text)
print(sentences)

### 2. Tokenization
Split them into individual words! If this is too easy go look up tokenization in Japanese which cannot rely on spaces to split the words. -> Search 'MeCab Japanese'

In [ ]:
words = nltk.word_tokenize(text)
print(words)

Seems like we haven't kept the work we did in the sentence segmentation section - how should we apply word tokenization on top of sentence segmentation? Analysing sentences one at a time is usually preferable to minimize complexity.

In [ ]:
# Write your code here!

### 3. Part-of-speech tagging
Now we want to analyse the first sentence to see what parts of speech are present. Usually we can use just the nouns present to guess what is going on in a sentence.

In [ ]:
words = nltk.word_tokenize(sentences[0])
print(f'First sentence tokenized: {words}')
pos_tags = nltk.pos_tag(words)
print(f'Part of speech tags: {pos_tags}')

### 4. Normalization
We want to reduce vocabulary size which will help improve our NLP.
#### 4.1 Stemming
Simply truncating words to their stems (may not be words themselves). Let's try 2 different stemmers over some examples. Snowball stemmer is the 'upgraded' version of the Porter stemmer, including the ability for users to not process stopwords as sometimes the conjugated forms may have different meanings, e.g. 'to be' and 'a being'.

In [ ]:
# Define your own word list
word_list = ['being', 'a', 'fairly', 'mischievous', 'cat', 'causing', 'trouble', 'late', 'into', 'the', 'night']

In [ ]:
porter = nltk.stem.PorterStemmer()
snowball = nltk.stem.SnowballStemmer('english', ignore_stopwords = True)

print([porter.stem(word) for word in word_list])
print([snowball.stem(word) for word in word_list])

#### 4.2 Lemmatisation
Another option is to reduce words to their base lemmas to 'standardize' words into their synonyms. Lemmatisation is better usually as it is more informative and uses PoS. Let's run the same sentence we stemmed in our lemmatizer.

In [ ]:
# Function to convert our PoS tags from one type to another used by wordnet
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return nltk.corpus.wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return nltk.corpus.wordnet.VERB
    elif treebank_tag.startswith('N'):
        return nltk.corpus.wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return nltk.corpus.wordnet.ADV
    else:
        return None


In [ ]:
lemmatizer = nltk.stem.WordNetLemmatizer()
pos_tags = nltk.pos_tag(word_list) # we need the part of speech tags for lemmatisation
print(pos_tags)

lemmatized_words = [lemmatizer.lemmatize(pos_tag[0], get_wordnet_pos(pos_tag[1])) if get_wordnet_pos(pos_tag[1]) else pos_tag[0] for pos_tag in pos_tags]
print(lemmatized_words)

### 5. Removing stopwords
We remove stopwords to get rid of noise that is usually irrelevant.

In [ ]:
stop_words = nltk.corpus.stopwords.words('english')
without_stop_words = [word for word in lemmatized_words if not word in stop_words]
print(without_stop_words)

## B. Regex and Fuzzy Matching
Let's import some packages we will be using!

In [ ]:
import re
from fuzzywuzzy import fuzz
! pip install pyspellchecker
from spellchecker import SpellChecker

### 1. Regex
Let's try regular expression matching

In [ ]:
print(re.sub('a', 'b', 'abacadabra is a magic spell'))
print(re.findall(r'[A-Z]+', 'I am feeling okay but if I am happy I ONLY USE CAPS'))

Okay, your turn. Please Google to look up new expressions! regex101.com is great for trying out expressions too

Task 1: fill in the regex expression to match only on ['1YESa', 'aYES3', 'asdkfYES', 'YES'].

In [ ]:
task_1 = '1YESa 2yesb aYES3 asdkfYES YES yES6'
print(re.findall(r'[a-zA-Z0-9]*YES[a-zA-Z0-9]*', task_1))

Task 2: fill in the regex expression to match all 6 e-mail addresses in the string

In [ ]:
task_2 = 'Hi Sally, Could you please forward this e-mail to the e-mails in this list please? l&d@deloitte.co.uk, nlp101@deloitte.co.uk and aitesting@deloitte.com, ai101@deloitte.com and ukai@deloitte.com, l&d@deloitte.com Thanks! My instagram handle is @swiftnlp.'
print(task_2, '\n')
print(re.findall(r'[A-Za-z0-9&]+\@[A-Za-z]+(?:\.[A-Za-z]+)+', task_2))

### 2. Fuzzy Matching

Quick method for matching strings that are similar but not exactly the same - great for data cleaning!

In [ ]:
string1 = 'Online NLP 101 Course for Everyone!'
string2 = 'Course in NLP'

In [ ]:
print(fuzz.ratio(string1, string2))
print(fuzz.partial_ratio(string1, string2))
print(fuzz.token_sort_ratio(string1, string2))
print(fuzz.token_set_ratio(string1, string2))

Another use case for fuzzy matching is simple spell checking using pyspellchecker which relies on Levenshtein distance but also the frequency the word appears in the English language. You can then replace these spelling errors!

In [ ]:
spell = SpellChecker()

# find those words that may be misspelled
misspelled = spell.unknown(['something', 'is', 'hapenning', 'here'])

for word in misspelled:
    # Get the one `most likely` answer
    print('most likely candidate: ', spell.correction(word))

    # Get a list of `likely` options
    print('all likely candidates: ', spell.candidates(word))

## C. ML & DL NLP
### 1. Named Entity Recognition
Let us use a popular NLP package named Spacy which will also do all the pre-processing/Classic NLP steps above!

In [ ]:
import spacy
from spacy import displacy
! pip install contextualSpellCheck
import contextualSpellCheck

In [ ]:
nlp = spacy.load("en_core_web_sm")
doc = nlp(text)

displacy.render(doc, style="ent")

### 2. BERT use-case: Contextual spell checking
Let's use spacy for contextual spell checking too!

In [ ]:
contextualSpellCheck.add_to_pipe(nlp)

In [ ]:
doc = nlp('Income was $9.4 milion conpar to the prior year of $2.7 milion.')
doc._.outcome_spellCheck

### 3. Sentiment Analysis

Let us use NLTK's built-in, pretrained sentiment analyser. It is called VADER. VADER is best suited for social media language - short w/ abbreviations and slang.

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
sia.polarity_scores("Wow, NLTK is really powerful!")

## D. Build your own ML model for classification

Import all required packages

In [ ]:
!pip install contractions
import contractions
import string
from itertools import chain
from nltk.corpus import movie_reviews as mr
import pandas as pd
import sklearn
from sklearn import naive_bayes, linear_model, ensemble, svm

Load in data

In [ ]:
positive_review_ids = nltk.corpus.movie_reviews.fileids(categories=["pos"])
negative_review_ids = nltk.corpus.movie_reviews.fileids(categories=["neg"])
pos_revs = pd.DataFrame([nltk.corpus.movie_reviews.raw(rev) for rev in positive_review_ids], columns = ['review'])
neg_revs = pd.DataFrame([nltk.corpus.movie_reviews.raw(rev) for rev in negative_review_ids], columns = ['review'])
pos_revs['positive_sentiment'] = 1
neg_revs['positive_sentiment'] = 0
all_revs = pos_revs.append(neg_revs)
all_revs.head(5)

Apply cleaning steps here

In [ ]:
# Write cleaning code here

Split into train and test set for model training

In [ ]:
train_x, valid_x, train_y, valid_y = sklearn.model_selection.train_test_split(all_revs['review'], all_revs['positive_sentiment'])

Vectorize using 2 methods: count vectorization and tfidf vectorization

In [ ]:
count_vect = sklearn.feature_extraction.text.CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(all_revs['review'])
tfidf_vect = sklearn.feature_extraction.text.TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features = 500)
tfidf_vect.fit(all_revs['review'])

xtrain_count = count_vect.transform(train_x)
xvalid_count = count_vect.transform(valid_x)

xtrain_tfidf = tfidf_vect.transform(train_x)
xvalid_tfidf = tfidf_vect.transform(valid_x)

Let's try a couple of classic classification models:
1. Naive Bayes
2. Linear Model (Logistic Regression)
3. Support Vector Machines
4. Random Forest

In [ ]:
# Function created to make it easier to train various models/algorithms
def train_model(classifier, feature_vector_train, label, feature_vector_valid):
    classifier.fit(feature_vector_train, label)
    predictions = classifier.predict(feature_vector_valid)
    return sklearn.metrics.classification_report(valid_y, predictions)

In [ ]:
nb_cv = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
print("Naive Bayes + Count Vectors: ", nb_cv)

nb_tfidf = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
print("Naive Bayes + TFIDF Vectors: ", nb_tfidf)

lr_cv = train_model(linear_model.LogisticRegression(), xtrain_count, train_y, xvalid_count)
print("Logistic Regression + Count Vectors: ", lr_cv)

lr_tfidf = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print("Logistic Regression + TFIDF Vectors: ", lr_tfidf)

svm_cv = train_model(svm.SVC(), xtrain_count, train_y, xvalid_count)
print("Support Vector Machines + Count Vectors: ", svm_cv)

svm_tfidf = train_model(svm.SVC(), xtrain_tfidf, train_y, xvalid_tfidf)
print("Support Vector Machines + TFIDF Vectors: ", svm_tfidf)

rf_cv = train_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xvalid_count)
print("Random Forest + Count Vectors: ", rf_cv)

rf_tfidf = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf, train_y, xvalid_tfidf)
print("Random Forest + TFIDF Vectors: ", rf_tfidf)

The challenge is now to improve the sentiment predictor. There are many methods but the easiest one is by preprocessing the text.

If you don't know where to start, try:
1. Removing contractions
2. Removing stopwords
3. Lemmatising the text

After your text is clean and processed, you can then start looking at adding additional features (e.g. word count), optimizing the vectorization methods and classification algorithms.

## Thank you!
Thank you for joining NLP 101 and I hope you have learnt something useful today! Please feel free to contact Michelle (miclai@deloitte.co.uk) if you have any further questions :)

Here is how someone else built their own sentiment model with the same dataset:
https://realpython.com/python-nltk-sentiment-analysis/#customizing-nltks-sentiment-analysis

If you want to try build something more sophisticated, here is a good example with a Kaggle dataset:
https://mlwhiz.com/blog/2020/05/24/multiteexamplextclass/